In [44]:
import numpy as np
import random, math
import copy

Dimension = 3
qubits = 2
A = np.array([[1, -1, 1], [-2, 3, 4], [1, -4, -1]]) # Row vectors
b = ([4, -4, 8]) # Column vectors

# 9 qubits with integer solutions
# x1 = q1 + 2q2 - 4q3
# x = {{1}, {-2}, {1}}
# QUBO minimum -96

QM = np.zeros(((qubits+1)*Dimension, (qubits+1)*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        cef1 = pow(2,2*qubits)*pow(A[k][i],2)
        cef2 = pow(2,qubits+1)*A[k][i]*b[k]
        po2 = (qubits+1)*i + qubits 
        QM[po2][po2] = QM[po2][po2] + cef1 + cef2
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = (qubits+1)*i + l
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l in range (qubits):
            qcef = pow(2, l+qubits+1)*pow(A[k][i],2)
            po3 = (qubits+1)*i + l
            po4 = (qubits+1)*i + qubits 
            QM[po3][po4] = QM[po3][po4] - qcef
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = (qubits+1)*i + l1
                po2 = (qubits+1)*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                
### Second quadratic term ###                 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            qcef = pow(2, 2*qubits+1)*A[k][i]*A[k][j]
            po3 = (qubits+1)*i + qubits
            po4 = (qubits+1)*j + qubits
            QM[po3][po4] = QM[po3][po4] + qcef
            for l in range (qubits):
                qcef = pow(2, l+qubits+1)*A[k][i]*A[k][j]
                po5 = (qubits+1)*i + qubits 
                po6 = (qubits+1)*j + l
                QM[po5][po6] = QM[po5][po6] - qcef
                po7 = (qubits+1)*i + l
                po8 = (qubits+1)*j + qubits
                QM[po7][po8] = QM[po7][po8] - qcef
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = (qubits+1)*i + l1
                    po2 = (qubits+1)*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef


# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range((qubits+1)*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",(qubits+1)*Dimension,"','q",(qubits+1)*Dimension,"'):",format(QM[(qubits+1)*Dimension-1][(qubits+1)*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range((qubits+1)*Dimension-1):
    for j in range(i+1,(qubits+1)*Dimension):
        qdrt1 = i + 1
        qdrt2 = j + 1
        if i == (qubits+1)*Dimension-2 and j == (qubits+1)*Dimension-1:
            print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
        else:
            print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[ -34.   24.  -48.  -22.  -44.   88.  -16.  -32.   64.]
 [   0.  -56.  -96.  -44.  -88.  176.  -32.  -64.  128.]
 [   0.    0.  256.   88.  176. -352.   64.  128. -256.]
 [   0.    0.    0.  122.  104. -208.   30.   60. -120.]
 [   0.    0.    0.    0.  296. -416.   60.  120. -240.]
 [   0.    0.    0.    0.    0.   32. -120. -240.  480.]
 [   0.    0.    0.    0.    0.    0.   58.   72. -144.]
 [   0.    0.    0.    0.    0.    0.    0.  152. -288.]
 [   0.    0.    0.    0.    0.    0.    0.    0.  128.]]

Minimum energy is  -96


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-34.0, ('q2','q2'):-56.0, ('q3','q3'):256.0, ('q4','q4'):122.0, ('q5','q5'):296.0, ('q6','q6'):32.0, ('q7','q7'):58.0, ('q8','q8'):152.0, ('q9','q9'):128.0}

quadratic = {('q1','q2'):24.0, ('q1','q3'):-48.0, ('q1','q4'):-22.0, ('q1','q5'):-44.0, ('q1','q6'):88.0, ('q1'

In [45]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-34.0, ('q2','q2'):-56.0, ('q3','q3'):256.0, ('q4','q4'):122.0, ('q5','q5'):296.0, ('q6','q6'):32.0, ('q7','q7'):58.0, ('q8','q8'):152.0, ('q9','q9'):128.0}

quadratic = {('q1','q2'):24.0, ('q1','q3'):-48.0, ('q1','q4'):-22.0, ('q1','q5'):-44.0, ('q1','q6'):88.0, ('q1','q7'):-16.0, ('q1','q8'):-32.0, ('q1','q9'):64.0, ('q2','q3'):-96.0, ('q2','q4'):-44.0, ('q2','q5'):-88.0, ('q2','q6'):176.0, ('q2','q7'):-32.0, ('q2','q8'):-64.0, ('q2','q9'):128.0, ('q3','q4'):88.0, ('q3','q5'):176.0, ('q3','q6'):-352.0, ('q3','q7'):64.0, ('q3','q8'):128.0, ('q3','q9'):-256.0, ('q4','q5'):104.0, ('q4','q6'):-208.0, ('q4','q7'):30.0, ('q4','q8'):60.0, ('q4','q9'):-120.0, ('q5','q6'):-416.0, ('q5','q7'):60.0, ('q5','q8'):120.0, ('q5','q9'):-240.0, ('q6','q7'):-120.0, ('q6','q8'):-240.0, ('q6','q9'):480.0, ('q7','q8'):72.0, ('q7','q9'):-144.0, ('q8','q9'):-288.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc. chain_b.
2    1  0  0  0  1  1  1  0  0  -96.0      10      0.0
3    0  0  0  0  1  1  1  0  0  -90.0      10      0.0
4    0  1  0  0  1  1  1  0  0  -90.0      14      0.0
5    1  1  0  1  1  1  1  0  0  -90.0      11      0.0
6    1  1  1  1  0  1  1  0  0  -90.0      25      0.0
7    0  0  0  0  1  1  0  0  0  -88.0       6      0.0
8    0  1  0  0  1  1  0  1  0  -88.0      19      0.0
9    0  0  0  1  0  1  1  0  0  -86.0      18      0.0
10   0  1  0  1  1  1  1  0  0  -86.0       3      0.0
11   1  1  0  0  1  1  0  1  0  -86.0      22      0.0
12   1  1  1  0  1  1  0  0  0  -86.0      12      0.0
13   1  0  0  1  0  1  0  1  0  -82.0      44      0.0
14   0  0  0  1  0  1  0  1  0  -82.0      45      0.0
15   0  1  0  1  1  1  0  0  0  -82.0       9      0.0
16   1  0  0  1  1  1  0  0  0  -82.0      16      0.0
17   0  1  1  1  0  1  1  0  0  -82.0       6      0.0
18   0  1  1  1  0  1  0  0  0  -82.0       5      0.0
109  0  0 